In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

output_dir = output_path / 'emissions'
# output_dir = Path('C:/Joanne_PSRC/temp_data/20250612_saves_person_results')

In [3]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

## Regional Emissions
Only includes light, medium, and heavy vehicles (bus vehicles are excluded)

In [4]:
emissions_summary = pd.read_csv(output_path / 'emissions/emissions_summary.csv')

network = load_network_summary(output_path / 'network/network_results.csv')

In [5]:
df_emissions_summary = emissions_summary.copy()

cols_dict = {'pollutant_name': 'Pollutant', 
             'veh_type': 'Vehicle Type',
             'start_tons': 'Start', 
             'intrazonal_tons': 'Intrazonal', 
             'interzonal_tons': 'Interzonal',
             'total_daily_tons': 'Total Daily (Tons)'}
cols = ['Start', 'Intrazonal','Interzonal', 'Total Daily (Tons)']
df_emissions_summary.rename(columns = cols_dict, inplace=True)


In [6]:
df = df_emissions_summary[df_emissions_summary['Vehicle Type'].isin(['light','medium','heavy'])].copy()
df = df.groupby('Pollutant').sum()
df.rename(columns = cols_dict, inplace=True)
df = df.loc[['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs']]

# FIXME line below is failing at 3.11. I dont see a need for it since there are no decimals in the output.
#df = df.applymap(lambda x: x if x > 100 else str(round(x,1)))
df[cols]

,Start,Intrazonal,Interzonal,Total Daily (Tons)
Pollutant,,,,
CO,6.9,0.2,19.8,26.8
NOx,0.7,0.0,3.5,4.2
PM25 Total,0.1,0.0,0.7,0.7
PM10 Total,0.1,0.1,4.9,5.0
CO2 Equivalent,163.9,23.3,"6,205.6","6,392.8"
VOCs,0.5,0.0,0.4,0.9


## Emissions by Vehicle Type

- VMT

In [7]:
df_network = network.copy()

df_network['@lveh'] = df_network[['@hov2_inc1','@hov2_inc2', '@hov2_inc3', 
                                  '@hov3_inc1', '@hov3_inc2', '@hov3_inc3',
                                  '@sov_inc1', '@sov_inc2', '@sov_inc3', 
                                  '@tnc_inc1', '@tnc_inc2','@tnc_inc3']].sum(axis=1)

df_network['light'] = df_network['@lveh']*df_network['length']
df_network['medium'] = df_network['@mveh']*df_network['length']
df_network['heavy'] = df_network['@hveh']*df_network['length']

index_labels = ['light','medium','heavy']
df = pd.DataFrame(index=index_labels)
df['VMT'] = df_network[index_labels].sum()

df.index.name = 'Vehicle Type'

df.to_csv(output_dir / 'vmt_by_vehtype_summary.csv')
df

,VMT
Vehicle Type,
light,"88,301,638.2"
medium,"3,562,815.6"
heavy,"3,536,657.0"


- Emissions

In [8]:
# Calculate emissions and VMT by vehicle type and save results
# Note that Total VMT will not match regional totals because we are not included buses in the emissions summaries

df = df_emissions_summary.copy()
df = df.groupby(['Pollutant','Vehicle Type']).sum()
df.rename(columns = cols_dict, inplace=True)

df = df.loc[['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs']][cols].copy()
df.to_csv(output_dir / 'emissions_by_vehtype_summary.csv')
df


Start  Intrazonal  Interzonal  Total Daily (Tons)
Pollutant      Vehicle Type                                                   
CO             heavy           0.0         0.0         3.8                 3.8
               light           5.7         0.1        14.9                20.7
               medium          1.2         0.0         1.1                 2.3
               transit         0.0         0.0         2.0                 2.1
NOx            heavy           0.0         0.0         2.9                 2.9
               light           0.4         0.0         0.3                 0.7
               medium          0.3         0.0         0.3                 0.6
               transit         0.0         0.0         0.2                 0.2
PM25 Total     heavy           0.0         0.0         0.1                 0.1
               light           0.1         0.0         0.5                 0.6
               medium          0.0         0.0         0.0                 0.1
               transit         0.0         0.0         0.0                 0.0
PM10 Total     heavy           0.0         0.0         0.5                 0.5
               light           0.1         0.1         4.1                 4.2
               medium          0.0         0.0         0.3                 0.3
               transit         0.0         0.0         0.1                 0.1
CO2 Equivalent heavy           1.9         2.3     3,281.9             3,286.1
               light         125.6        20.1     2,130.8             2,276.5
               medium         36.3         0.9       792.9               830.2
               transit         0.8         0.0       477.0               477.8
VOCs           heavy           0.0         0.0         0.0                 0.1
               light           0.3         0.0         0.3                 0.6
               medium          0.2         0.0         0.0                 0.2
               transit         0.0         0.0         0.0                 0.0